In [1]:
# Need help identifying the environment from your current notebook?
#!python -m ensurepip --default-pip
#!python -m pip --version
import sys
sys.executable
!which python3
!which pip
!pip -V

/opt/homebrew/bin/python3
/Users/p0s07cu/.code-puppy-venv/bin/pip
pip 25.2 from /Users/p0s07cu/.code-puppy-venv/lib/python3.13/site-packages/pip (python 3.13)


In [ ]:
# pip install google-adk
# This guarantees installation into the exact kernel environment.

# !{sys.executable} -m pip install google-adk

# !{sys.executable} -m pip install google-adk --break-system-packages

In [ ]:
# !pip install python-dotenv

# This guarantees installation into the exact kernel environment.
# !{sys.executable} -m pip install python-dotenv

In [ ]:
# !pip install yfinance --upgrade
# !python3 -m pip install yfinance --upgrade

In [ ]:
# !pip install feedparser

In [2]:
from dotenv import load_dotenv
import os

try:
    load_dotenv()
    api_key = os.getenv("GOOGLE_API_KEY")
    if api_key is None:
        raise Exception("GOOGLE_API_KEY is missing")
    print("✅ Gemini API key setup complete.")
    # print(api_key)
except Exception as e:
    print(f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}")


✅ Gemini API key setup complete.


In [23]:
from typing import Any, Dict
from google.adk.agents import Agent, SequentialAgent, LlmAgent
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner, InMemoryRunner
from google.adk.tools import google_search
from google.genai import types
from google.adk.models.google_llm import Gemini

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [8]:
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

In [ ]:
# First ADK Agent
root_agent = Agent(name="helpful_assistant", model="gemini-2.5-flash-lite", 
                   description="a simple assistant that can answer questions",
                   instruction="you are a helpful assistant. Use Google search for current info or if unsure.",
                   tools=[google_search])
print("Root Agent defined.")

In [ ]:
# Run your agent
runner = InMemoryRunner(agent=root_agent)
print("runner created.")

In [ ]:
# run in debug mode for prototyping
response = await runner.run_debug("What is Capstone project in Kaggle?")

In [ ]:
# multi agents system
# expense_analyzer
expense_analyzer_agent = Agent(name = "expanse_analyzer_assistant", model = "gemini-2.5-flash-lite",
                              description = "assitant which do analysis of expenses of a person",
                              instruction = "you are a smart anaylsis assistant. You analyze the expenses done by a person in a month.",
                              tools=[google_search],
                              output_key="result")
print("expense_analyzer_agent created.")

In [ ]:
# financial_advisor
financial_advisor_agent = Agent(name = "financial_advisor_assistant", model = "gemini-2.5-flash-lite",
                              description = "assitant who advisor on investment, savings & loan decision to a person",
                              instruction = "you are a smart advisor assistant. On the basis of result of expanse_analyzer_assistant, suggest good financial advise to a person.",
                              tools=[google_search],
                              output_key="advise")
print("financial_advisor_agent created.")

In [ ]:
# root agent - orchestrator
fin_track_agent = SequentialAgent(name = "finTrack", 
                                 sub_agents=[expense_analyzer_agent, financial_advisor_agent])

# Similarly you can create ParallelAgents

print("SequentialAgent fin_track_agent is created.")

In [ ]:
query = "Rahul got salary of 20,000/- today. He has pending bills on his credit cards of around 5,000/- but he decided to buy his favourite video game worth 18,000/-. What should Rahul do?"
print(query)

In [ ]:
runner = InMemoryRunner(agent=fin_track_agent)
response = await runner.run_debug(query)

In [ ]:
# Custom Function Tools -> you own code, solves domain specifi problems, your control over functionality
def risk_assesment(expense_type: str) -> str :
    """Take the expense type and decide the risk invloved with the expense.
    Args: 
        expense_type: String which describes the type of expense like rent, loan, bills, etc.
    Returns:
        String with the resultant risk as evaluated by the internal logic
    """
    if expense_type == "rent":
        return "NECESSITY"
    else:
        return "NON_NECESSITY"

In [ ]:
print(risk_assesment("rent"))

In [ ]:
risk_assesser_agent = LlmAgent(name = "risk_assesment_agent",
                              model = Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
                              instruction = "You are a smart risk assesment assistant, who identifies the risk from the given expenses of a person",
                              tools = [risk_assesment])
print("risk_assesser_agent is created")

In [ ]:
query = "I have to pay 10,000/- rent from 20,000/- salary. what is the risk level of this expense?"
print(query)

In [ ]:
risk_assesser_runner = InMemoryRunner(agent=risk_assesser_agent)
_ = await risk_assesser_runner.run_debug(query)

In [ ]:
# Agent Tool -> Agent act as a tool, ex. calcAgent which generated python code to do calculations

# Agents vs Sub-agents
# Agents => Delegation for specific tasks (like calculations)
# Sub Agents => Handoff to specialists (like customer support agent)

In [ ]:
# Create a MCP server and call it here
# Idea - MCP for news related to finance & market

In [ ]:
import feedparser
# can be used as tool functions
def fetch_yahoo_finance_news(limit=10):
    url = "https://finance.yahoo.com/rss/topstories"
    
    feed = feedparser.parse(url)
    
    articles = []
    for entry in feed.entries[:limit]:
        articles.append({
            "title": entry.title,
            "link": entry.link,
            "published": entry.published
        })
    
    return articles


# ▶️ Fetch and print
articles = fetch_yahoo_finance_news(limit=10)

for idx, a in enumerate(articles, start=1):
    print(f"\n{idx}. {a['title']}")
    print(f"   Link: {a['link']}")
    print(f"   Published: {a['published']}")


In [5]:
# Session Management
# LLMs are inherently stateless.
# Sessions -> Short term memory management
# Memory -> Long term memory

# Sessions: 
#     - single, continous conversations
#     - single user & agent, not shared with othr user
#     - session history for an Agent is not shared with other Agents
# Sessions 
#     1. Events - building blocks of conversation
#     2. State - Agents's scratchpad, global {key, value} pair storage

# Session Manager - SessionService - storage layer
# Runner - orchestation layer

# Session = A notebook 📓
# Events = Individual entries in a single page 📝
# SessionService = The filing cabinet storing notebooks 🗄️
# Runner = The assistant managing the conversation 🤖


In [32]:
# a user logged in app
# starts its query
APP_NAME = "capstone"
USER_ID = "prasri"
SESSION = "Monday_17nov"

MODEL_NAME = "gemini-2.5-flash-lite"

root_agent = Agent(model =Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
                   name="fin_trac_agent", description="A smart finance tracker assistant")

session_service = InMemorySessionService()

runner = Runner(agent=root_agent, app_name=APP_NAME, session_service=session_service)
print("stateful agent created!")

stateful agent created!


In [30]:
# Define helper functions that will be reused throughout the notebook
async def run_session(
    runner_instance: Runner,
    user_queries: list[str] | str = None,
    session_name: str = "default",
):
    print(f"\n ### Session: {session_name}")

    # Get app name from the Runner
    app_name = runner_instance.app_name

    # Attempt to create a new session or retrieve an existing one
    try:
        session = await session_service.create_session(
            app_name=app_name, user_id=USER_ID, session_id=session_name
        )
    except:
        session = await session_service.get_session(
            app_name=app_name, user_id=USER_ID, session_id=session_name
        )

    # Process queries if provided
    if user_queries:
        # Convert single query to list for uniform processing
        if type(user_queries) == str:
            user_queries = [user_queries]

        # Process each query in the list sequentially
        for query in user_queries:
            print(f"\nUser > {query}")

            # Convert the query string to the ADK Content format
            query = types.Content(role="user", parts=[types.Part(text=query)])

            # Stream the agent's response asynchronously
            async for event in runner_instance.run_async(
                user_id=USER_ID, session_id=session.id, new_message=query
            ):
                # Check if the event contains valid content
                if event.content and event.content.parts:
                    # Filter out empty or "None" responses before printing
                    if (
                        event.content.parts[0].text != "None"
                        and event.content.parts[0].text
                    ):
                        print(f"{MODEL_NAME} > ", event.content.parts[0].text)
    else:
        print("No queries!")


print("✅ Helper functions defined.")

✅ Helper functions defined.


In [33]:
# test sessions
await run_session(runner, 
                  [
                      "Hi, My name is Prakhar and I spent 15000 in shopping.",
                      "Who am I and how much I spent?"
                  ],
                  "stateful-agentic-session")


 ### Session: stateful-agentic-session

User > Hi, My name is Prakhar and I spent 15000 in shopping.
gemini-2.5-flash-lite >  Hi Prakhar! Thanks for letting me know about your shopping expense.

To help me track this for you, could you tell me:

*   **When** did you spend this 15000? (e.g., today, yesterday, last week)
*   Do you have any **specific categories** in mind for this shopping? (e.g., clothing, electronics, groceries, household items)

The more details you can provide, the better I can organize your finances!

User > Who am I and how much I spent?
gemini-2.5-flash-lite >  You are **Prakhar**, and you have spent **15000** on shopping so far.
